
---

# 🇺🇦 ZNO Solver: How I survived "Offline Mode"

**Task:** Solve Ukrainian ZNO exams.
**Constraint:** No Internet. No API calls.
**My Hardware:** A Kaggle GPU that cries if you look at it wrong.

Basically, I couldn't just call GPT-4o. I had to make a small, dumb model (Qwen 7B) act like a smart model, all while cut off from the outside world.

---

## Phase 1: The "Professor" (Cheating with Gemini)

Qwen 2.5 7B is decent, but on Ukrainian History or Literature, it mostly just guesses "B" and prays. I needed it to actually *think*.

Since I could use the internet before the submission, I used **Gemini 3.0 Flash Preview** to generate a "Reasoning Dataset." I fed it 3k raw questions and forced it to explain why an answer is right.

**The Prompt I used:**

```python
prompt = f"""
You are a ZNO tutor.
Question: {q}
Options: {opts}
Task: Explain the logic in 1-2 sentences. End with "Відповідь: [Letter]".
"""

```

**Result:** I got a dataset where the AI explains that *"Panas Myrny wrote 'Hiba revut voly...', so the answer is A"* instead of just saying *"A"*.

---

## Phase 2: The "Student" (Fine-Tuning)

I took that data and force-fed it to Qwen using **QLoRA** (because I don't have enough VRAM to train the full model).

**Snippet from `train.py`:**

```python
# LoRA Config because I am poor (VRAM-wise)
peft_config = LoraConfig(
    r=16, lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # Hit everything
    task_type="CAUSAL_LM"
)

```

It trained for about an hour. The loss went down. I felt like a genius. (Spoiler: I was not).

---

## Phase 3: The Nightmare (Inference)

This is where everything went wrong.

### Attempt 1: "Just ask it nicely" (Generation)

I tried to make the model generate the explanation like I trained it to.

**The Prompt:**
`P: <Question>... Answer this.`
**The Output:**
`A: Analysis: Taras Shevchenko was a great artist. He painted many things. The answer is... [Model continues writing for 500 tokens about Shevchenko's life] ... Answer: B.`

**The Problem:**

1. It wouldn't shut up.
2. **Hallucinations:** It would write a perfectly logical explanation for why the earth is flat and then pick the wrong letter.
3. **Parsing Hell:** I tried Regex to find the letter. It failed 30% of the time because the model wrote "Answer is (A)" or "Відповідь - А" or "I think A".

**Score:** Terrible.

---

### Attempt 2: "Math over Vibes" (Logit Scoring)

I read a paper (`2025.unlp-1.2.pdf`) that said: *Don't let the model talk. Just look at the math.*

Instead of generating text, I checked the probability of the next token being 'А', 'Б', 'В', 'Г', or 'Д'.

**The Code:**

```python
# Force the model to look at the math
with torch.inference_mode():
    outputs = model(**inputs)
    logits = outputs.logits[:, -1, :] # Last token
    scores = logits[:, [id_A, id_B, id_C, id_D, id_E]]
    winner = torch.argmax(scores)

```

**The Mistake:**
I got `CUDA OutOfMemory Error` immediately.
I tried `BATCH_SIZE = 32`. **Crash.**
I tried `BATCH_SIZE = 8`. **Crash.**

**The Fix:**
`BATCH_SIZE = 1`. Yes, it took 2 hours to run. But it didn't crash.

---

### Attempt 3: The "Bias" Problem

My logit scorer was working, but I noticed it picked "A" or "B" way too often.
Why? Because the model is lazy.

**The Solution: Few-Shot Prompting**
I added 4 examples to the prompt before the real question.

* Example 1 -> Answer A
* Example 2 -> Answer B
* Example 3 -> Answer G
* Example 4 -> Answer D

This forced the model to "remember" that other letters exist.

**Final Code Snippet (The one that actually worked):**

```python
# The "Nuclear Option" Prompt
FEW_SHOT = """
...Example 1 (Ans: A)...
...Example 2 (Ans: B)...
...Example 3 (Ans: G)...
...Example 4 (Ans: D)...
"""

instruction = "Дай відповідь буквою-варіантом."
final_prompt = f"{instruction}\n\n{FEW_SHOT}\n\nQuestion: {q}\nAnswer:"

```

---

## The "Offline" Struggle

You don't appreciate `pip install` until it's gone.
I had to download every library on my laptop, zip them, upload them to Kaggle, and install them like this:

```python
# I hate this part
!pip install --no-index --find-links=offline_libs bitsandbytes peft transformers

```

**Fun error I got:**
`ModuleNotFoundError: validate_bnb_backend_availability`
*Translation:* Your bitsandbytes version hates your transformers version.
*Fix:* Uninstalled everything, reinstalled in specific order. Cried a little.

---

## Final Results

* **Random Guessing:** ~20%
* **Text-Only Baseline:** ~34%
* **My Score:** **~38%**

Is it perfect? No.
Did I beat the baseline without using RAG or the Internet? **Yes.**

We hit a "Knowledge Ceiling." The model is small (7B). It just doesn't know some specific facts about Ukrainian history, and no amount of prompt engineering can fix missing neurons. To get >50%, we'd need RAG (searching a textbook), but for a pure model approach, this is the limit.